# Generación de datos sintéticos con Fully conditional specification 

En este cuaderno se muestra un ejemplo de creación de datos sintéticos siguiendo el método de *fully conditional specification* en el lenguaje R.

## Instalación de bibliotecas

En este ejemplo se utilizará la biblioteca de R synthpop la cual está específicamente creada para la creación y evaluación de datos sintéticos. También se hace uso de la biblioteca minio.s3 para el acceso al lago de datos y la biblioteca de tydiverse para manejar los datos. 

In [ ]:
library(devtools)
install_github("nagdevAmruthnath/minio.s3")
install.packages("synthpop")
install.packages("tidyverse")

## Importación de bibliotecas

In [ ]:
library(minio.s3)
library(dplyr)
library(readr)
library(forcats)
library(stringr)
library(synthpop)

## Variables de acceso al lago 

En la siguiente celda se establecen variables para acceder al lago de datos. Es necesario tener accesos adecuados para obtener los datos. 

In [ ]:
Sys.setenv("AWS_ACCESS_KEY_ID" = readline(prompt = "Usuario: "),
           "AWS_SECRET_ACCESS_KEY" = getPass(prompt = "Contraseña: "),
          "AWS_S3_ENDPOINT"= "hostname:port"
          )
ruta_archivo <- "s3://carpeta/datos.ext"

## Acceso a los datos del lago

Para este ejercicio se obtuvieron datos de la Encuesta de Ocupación y Empleo. Estos difieren de los datos disponibles públicamente ya que contienen información real de localidad y manzana que son omitidas por cuestiones de privacidad. 

Debido a la naturaleza sensible de estos datos, el acceso a esta información es controlado solo a usuarios autorizados. Así también se omitirá desplegar información particular de los datos originales.

In [4]:
tabla_sdem  <- as_tibble(s3read_using(read_csv, col_types =cols(.default ="c"), object=ruta_archivo, opts = list(use_https = F)))

## Variables de interés 

Para este ejercicio no requerimos hacer uso de FAC el cual indica el factor de expansión. Por lo que las variables de interés serán:

- **LOC:** Localidad
- **MUN:** Municipio
- **T_LOC:** Tamaño de localidad 
- **MAN:** Manzana
- **CD_A:** Ciudad auto representada
- **ENT:** Entidad
- **AGEB:** Área GeoEstadística Básica
- **SEX:** Sexo
- **EDA:** Edad 
- **NAC_DIA:** Dia de nacimiento
- **NAC_MES:** Mes de nacimiento
- **NAC_ANIO:** Año de nacimiento 
- **CS_P13_1:** Nivel escolar
- **POS_OCU:** Posición en la ocupación
- **INGOCUP:** Ingreso del personal ocupado 

Como se puede observar, las variables son en general variables de identificación o variables sensibles.

In [ ]:
variables_interes <- c('LOC','MUN','T_LOC','MAN','CD_A','ENT','AGEB','SEX','EDA','NAC_DIA','NAC_MES','NAC_ANIO','CS_P13_1','POS_OCU','INGOCUP')
variables_interes

La tabla original se limita a las variables de interés, dentro del mismo proceso se elige el tipo de variables. 
La mayoría de los datos son tipo factor. Debido a que representan un numero finito de clases donde no tiene sentido analizar el valor numérico.
La excepción resulta las variables EDA, NAC_DIA, NAC_MES, NAC_ANIO, INGOCUP las cuales si hacen referencia a un valor numérico. 

Adicionalmente se realiza un muestreo aleatorio con solo el 1% de los datos, esto permite obtener resultados más rápidos. Una vez establecido el modelo final, es necesario utilizar la tabla de datos completa.

In [6]:
# Establecemos una semilla para que los resultados sean reproducibles
set.seed(2022)
sdem_interes  <- tabla_sdem %>% 
    select(all_of(variables_interes))  %>%
    mutate_at( 
        c(
            'LOC','MUN','T_LOC','MAN','CD_A','ENT','AGEB','SEX','CS_P13_1','POS_OCU'
        ),
        factor
    ) %>% 
    mutate_at( 
        c(
            'EDA','NAC_DIA','NAC_MES','NAC_ANIO','INGOCUP'
        ),
        as.numeric
    ) %>% 
    slice_sample(prop=0.01,replace=FALSE)

Revisamos las variables de la tabla para identificar de manera temprana problemas. Revisamos que tengan la clase correcta y el número de faltantes sea correcto.

In [ ]:
var_data <- codebook.syn(sdem_interes)
var_data$tab

## Síntesis

El proceso de síntesis necesita adaptarse específicamente. La manera más sencilla de hacer esto es realizar una síntesis inicial sin crear registros. 
Con esto obtenemos las estructuras necesarias para adaptar el proceso, en las siguientes celdas se modificarán para nuestras necesidades.


In [ ]:
system.time( syn_base <- syn(sdem_interes, m= 0, maxfaclevels = 6000) )
syn_base

### Secuencia de visita 

La manera que FCS funciona es que va modelando una variable a la vez. Por lo que es importante establecer el orden en que las columnas son sintetizadas. 
Comenzando con las variables "independientes" y después las variables que dependen de las primeras y así sucesivamente.

In [ ]:
my_visitsequence <- syn_base$visit.sequence
my_visitsequence

In [ ]:
my_visitsequence <- names( c(
    my_visitsequence["ENT"]
    ,my_visitsequence["CD_A"]
    ,my_visitsequence["MUN"]
    ,my_visitsequence["LOC"]
    ,my_visitsequence["T_LOC"]
    ,my_visitsequence["AGEB"]
    ,my_visitsequence["MAN"]
    ,my_visitsequence["SEX"]
    ,my_visitsequence["EDA"]
    ,my_visitsequence["NAC_DIA"]
    ,my_visitsequence["NAC_MES"]
    ,my_visitsequence["NAC_ANIO"]
    ,my_visitsequence["CS_P13_1"]
    ,my_visitsequence["POS_OCU"]
    ,my_visitsequence["INGOCUP"]
)
)
my_visitsequence

### Métodos de modelaje

Es posible establecer el método con el que se va a modelar cada variable respecto a la naturaleza. 
Algunas variables solo necesitan ser elegidas aleatoriamente ya que no son predichas por otras variables por lo que 'sample' puede ser suficiente.

El método por defecto es *classification and regression trees* (CART) el cual entrega buenos resultados, sin embargo, puede tener tiempos largos, sobre todo si el número de clases predictoras es bastante alto. 

El método de *bag* utiliza bagging con randmom forest, se encontró que este método es mucho más rápido que CART por lo que es usado cuando las relaciones entre variables son simples. Por ejemplo, la ciudad autorepresentada depende de la entidad pero la regla de decisión es bastante simple, solo se pueden elegir ciudades que están dentro de la entidad.

In [ ]:
my_method <- syn_base$method
my_method

In [ ]:
my_method["ENT"] <- "sample"
my_method["CD_A"] <- "bag"
my_method["MUN"] <- "bag"
my_method["LOC"] <- "bag"
my_method["T_LOC"] <- "" # No se calcula ya que es una regla directa de la localidad
my_method["AGEB"] <- "sample" # AGEB parece no depender de variables geograficas 
my_method["MAN"] <- "sample" # MANZANA parece no depender de variables geograficas 
my_method["SEX"] <- "sample" # se toma sexo como no relacionada con otra variable
my_method["EDA"] <- "sample" # se toma edad como no relacionada con otra variable
my_method["NAC_DIA"] <- "sample" # se toma dia de nacimiento como no relacionada con otra variable
my_method["NAC_MES"] <- "sample" # se toma mes de nacimiento como no relacionada con otra variable
my_method["NAC_ANIO"] <- "cart"
my_method["CS_P13_1"] <- "cart"
my_method["POS_OCU"] <- "cart"
my_method["INGOCUP"] <- "cart"

my_method

### Matriz de predicciones 

La matriz de predicciones indica que variables fungirán como predictoras para otras variables. Por defecto, la primera variable no depende de ninguna otra, las variables n subsecuentes dependen de las n-1 variables ya sintetizadas. 

Esta aproximación es subóptima ya que algunas variables no tienen relación entre si e incluirlas puede aumentar la complejidad del modelo en gran medida.

In [ ]:
my_predictormatrix <- syn_base$predictor.matrix
my_predictormatrix

In [ ]:
# Eliminamos todos los predictores para que sean establecidos de manera explicita
my_predictormatrix[,] <- 0

my_predictormatrix["CD_A",c('ENT')] <- 1
my_predictormatrix["MUN",c('ENT','CD_A')] <- 1
my_predictormatrix["LOC",c('ENT','CD_A')] <- 1 # no nos permite a nivel municipio randomForest.default(m, y, ...): Can not handle categorical predictors with more than 53 categories.
#my_predictormatrix["T_LOC",c('ENT','MUN','LOC')] <- 1 
my_predictormatrix["NAC_ANIO",c('EDA','NAC_MES')] <- 1
my_predictormatrix["CS_P13_1",c('T_LOC','SEX','EDA')] <- 1
my_predictormatrix["POS_OCU",c('T_LOC','SEX','EDA',"CS_P13_1")] <- 1
my_predictormatrix["INGOCUP",c('T_LOC','SEX','EDA','POS_OCU')] <- 1

my_predictormatrix

### Modelo final

Una vez que adaptamos los parámetros, procedemos a realizar la síntesis de la tabla.

Note que se establece cont.na para identificar valores que representan información no disponible (NA). Esto permite dar contexto a los algoritmos de clasificación y regresión.

In [ ]:
system.time( 
    sdem_interes_syn <- syn(
        sdem_interes,
        method = my_method,
        visit.sequence = my_visitsequence,
        predictor.matrix = my_predictormatrix,
        maxfaclevels = 600,
        cont.na = list(EDA=99 ,NAC_DIA  = 99, NAC_MES = 99, NAC_ANIO = 9999 ),
        seed = 2022
    ) 
)

In [ ]:
sdem_interes_syn

In [ ]:
summary(sdem_interes_syn)

## Evaluación del modelo

La biblioteca synthpop también ofrece herramientas para evaluar las síntesis creadas. 
La más directa es comparar contra los modelos originales en conteos para cada una de las variables.

In [ ]:
compare(sdem_interes_syn, sdem_interes)